In [1]:
import pandas as pd
from etl.etl import etl
from gluonts.dataset.pandas import PandasDataset

# from gluonts.torch import DeepAREstimator
from gluonts.torch.model.deepar.estimator import DeepAREstimator
from utils.utils import read_config

pd.set_option("display.max_columns", None)

In [2]:
config = read_config("model")

In [3]:
df = etl()
df_train = df[df["date"] <= config["model"]["train_split"]]
df_test = df[df["date"] > config["model"]["train_split"]]

In [4]:
ds_train = PandasDataset.from_long_dataframe(
    dataframe=df_train,
    target="positive_rate",
    item_id="state_name",
    timestamp="date",
    static_feature_columns=["fema_region"],
    static_features=pd.DataFrame(
        pd.Categorical(sorted(list(df_train["fema_region"].unique()))),
        columns=["fema_region"],
    ),
    feat_dynamic_real=["inconclusive", "negative", "positive"],
)

Columns ['fema_region'] in static_features have 'object' as data type and will be ignored; consider setting this to 'category' using pd.DataFrame.astype, if you wish to use them as categorical columns.


In [5]:
[df_train["fema_region"].nunique()]

[10]

In [6]:
prediction_length = config["model"]["prediction_length"]
static_cat_cardinality = [df_train["fema_region"].nunique()]

estimator = DeepAREstimator(
    freq="D",
    prediction_length=prediction_length,
    num_layers=2,
    lr=1e-3,
    num_feat_dynamic_real=3,
    num_feat_static_real=0,
    num_feat_static_cat=1,
    cardinality=static_cat_cardinality,
    trainer_kwargs={"accelerator": "mps", "devices": 1, "max_epochs": 2},
)

In [7]:
type(estimator)

gluonts.torch.model.deepar.estimator.DeepAREstimator

In [8]:
predictor = estimator.train(training_data=ds_train)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/francoli/anaconda3/envs/covid-diagnostics-forecasting/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
/Users/francoli/anaconda3/envs/covid-diagnostics-forecasting/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping v

Epoch 0: : 50it [00:04, 10.92it/s, v_num=29, train_loss=-2.72]

Epoch 0, global step 50: 'train_loss' reached -2.72077 (best -2.72077), saving model to '/Users/francoli/Desktop/projects/covid-diagnostics-forecasting/lightning_logs/version_29/checkpoints/epoch=0-step=50.ckpt' as top 1


Epoch 1: : 50it [00:01, 25.77it/s, v_num=29, train_loss=-3.19]

Epoch 1, global step 100: 'train_loss' reached -3.18719 (best -3.18719), saving model to '/Users/francoli/Desktop/projects/covid-diagnostics-forecasting/lightning_logs/version_29/checkpoints/epoch=1-step=100.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: : 50it [00:01, 25.25it/s, v_num=29, train_loss=-3.19]


In [9]:
type(predictor)

gluonts.torch.model.predictor.PyTorchPredictor